In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
import math
import nltk
from nltk.translate.bleu_score import corpus_bleu
from rouge_score import rouge_scorer
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

# NLTK 데이터 다운로드 (처음 한 번만 실행)
nltk.download('punkt')

In [ ]:
# 1. 모델과 토크나이저 로드
model_name = './models/transformers/Qwen__Qwen2.5-7B-Instruct'  # 원하는 모델로 변경 가능
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
model.eval()

In [ ]:
# GPU 사용 가능 시 설정
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
model.to(device)


In [ ]:
# 2. 데이터 로드 (예: WikiText-2)
dataset = load_dataset('wikitext', 'wikitext-2-raw-v1', split='test')

In [ ]:
# 3. 퍼플렉서티 계산 함수
def calculate_perplexity(model, tokenizer, texts, batch_size=8):
    encodings = tokenizer(texts, return_tensors='pt', truncation=True, padding=True)
    input_ids = encodings.input_ids.to(device)
    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)
        # 손실(loss)은 평균 cross-entropy
        loss = outputs.loss
    perplexity = math.exp(loss.item())
    return perplexity

In [ ]:
# 4. BLEU 점수 계산 함수
def calculate_bleu(references, hypotheses):
    # NLTK의 corpus_bleu는 리스트의 리스트 형태를 요구
    references = [[ref.split()] for ref in references]
    hypotheses = [hyp.split() for hyp in hypotheses]
    bleu = corpus_bleu(references, hypotheses)
    return bleu

In [ ]:
# 5. ROUGE 점수 계산 함수
def calculate_rouge(references, hypotheses):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = [scorer.score(ref, hyp) for ref, hyp in zip(references, hypotheses)]
    # 평균 계산
    rouge1 = np.mean([s['rouge1'].fmeasure for s in scores])
    rouge2 = np.mean([s['rouge2'].fmeasure for s in scores])
    rougeL = np.mean([s['rougeL'].fmeasure for s in scores])
    return {'rouge1': rouge1, 'rouge2': rouge2, 'rougeL': rougeL}

In [ ]:
# 6. 예측 생성 함수 (예: 다음 문장 생성)
def generate_predictions(model, tokenizer, texts, max_length=50):
    inputs = tokenizer(texts, return_tensors='pt', truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=max_length, num_return_sequences=1)
    predictions = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
    return predictions

In [ ]:
# 7. 정확도 및 F1 스코어 계산 함수 (예: 텍스트 분류)
def calculate_classification_metrics(true_labels, predicted_labels):
    accuracy = accuracy_score(true_labels, predicted_labels)
    f1 = f1_score(true_labels, predicted_labels, average='weighted')
    return {'accuracy': accuracy, 'f1_score': f1}

In [ ]:
# 8. 평가 수행
# 예를 들어, 퍼플렉서티 계산
sample_texts = dataset['text'][:100]  # 샘플 데이터 사용
perplexity = calculate_perplexity(model, tokenizer, sample_texts)
print(f'퍼플렉서티: {perplexity}')

In [ ]:


# 텍스트 생성 및 BLEU, ROUGE 계산
# 참조 텍스트와 생성 텍스트가 필요합니다. 여기서는 예시로 동일한 텍스트를 사용
# 실제 사용 시에는 참조 텍스트와 모델이 생성한 텍스트를 준비해야 합니다.
references = sample_texts
hypotheses = generate_predictions(model, tokenizer, sample_texts)

bleu = calculate_bleu(references, hypotheses)
rouge = calculate_rouge(references, hypotheses)
print(f'BLEU 점수: {bleu}')
print(f'ROUGE 점수: {rouge}')

In [ ]:
# # 분류 태스크 예시 (참고: 실제 분류 태스크 데이터 필요)
# true_labels = [...]  # 실제 라벨
# predicted_labels = [...]  # 모델 예측 라벨
# metrics = calculate_classification_metrics(true_labels, predicted_labels)
# print(f'정확도: {metrics["accuracy"]}, F1 스코어: {metrics["f1_score"]}')